In [1]:
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
with open('credential.json') as f:
    data = json.load(f)
    client_id = data['client_id']
    client_secret = data['client_secret']

In [3]:
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

In [4]:
# Spotify object to access API
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [5]:
result = sp.search("John Mayer")

In [6]:
result['tracks']['items'][1]['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/0hEurMDQu99nJRq8pTxO14'},
  'href': 'https://api.spotify.com/v1/artists/0hEurMDQu99nJRq8pTxO14',
  'id': '0hEurMDQu99nJRq8pTxO14',
  'name': 'John Mayer',
  'type': 'artist',
  'uri': 'spotify:artist:0hEurMDQu99nJRq8pTxO14'}]

### Extract Spotify Albums

In [45]:
#Extract Artist's uri
artist_uri = result['tracks']['items'][1]['artists'][0]['uri']
#Pull all of the artist's albums
sp_albums = sp.artist_albums(artist_uri, album_type='album')
#Store artist's albums' names' and uris in separate lists
album_names = []
album_uris = []
for i in range(len(sp_albums['items'])):
    album_names.append(sp_albums['items'][i]['name'])
    album_uris.append(sp_albums['items'][i]['uri'])

In [46]:
album_names

['The Search for Everything',
 'Paradise Valley',
 'Born and Raised',
 'Battle Studies',
 'Where the Light Is: John Mayer Live In Los Angeles',
 'Continuum',
 'As/Is',
 'As/Is: Cleveland/Cincinnati, OH - 8/03-8/04/04',
 'As/Is: Houston, TX - 7/24/04',
 'As/Is: Philadelphia, PA/Hartford, CT - 8/14-8/15/04',
 'As/Is: Houston, TX - 7/24/04',
 'As/Is: Mountain View, CA - 7/16/04',
 'Heavier Things',
 'Any Given Thursday',
 'Any Given Thursday',
 'Room For Squares',
 'Inside Wants Out']

### Get Tracks

In [47]:
def albumSongs(uri):
    album = uri #assign album uri to a_name
    spotify_albums[album] = {} #Creates dictionary for that specific album
    #Create keys-values of empty lists inside nested dictionary for album
    spotify_albums[album]['album'] = [] #create empty list
    spotify_albums[album]['track_number'] = []
    spotify_albums[album]['id'] = []
    spotify_albums[album]['name'] = []
    spotify_albums[album]['uri'] = []
    tracks = sp.album_tracks(album) #pull data on album tracks
    for n in range(len(tracks['items'])): #for each song track
        spotify_albums[album]['album'].append(album_names[album_count]) #append album name tracked via album_count
        spotify_albums[album]['track_number'].append(tracks['items'][n]['track_number'])
        spotify_albums[album]['id'].append(tracks['items'][n]['id'])
        spotify_albums[album]['name'].append(tracks['items'][n]['name'])
        spotify_albums[album]['uri'].append(tracks['items'][n]['uri'])

In [48]:
spotify_albums = {}
album_count = 0
for i in album_uris: #each album
    albumSongs(i)
    print("Album " + str(album_names[album_count]) + " songs has been added to spotify_albums dictionary")
    album_count+=1 #Updates album count once all tracks have been added

Album The Search for Everything songs has been added to spotify_albums dictionary
Album Paradise Valley songs has been added to spotify_albums dictionary
Album Born and Raised songs has been added to spotify_albums dictionary
Album Battle Studies songs has been added to spotify_albums dictionary
Album Where the Light Is: John Mayer Live In Los Angeles songs has been added to spotify_albums dictionary
Album Continuum songs has been added to spotify_albums dictionary
Album As/Is songs has been added to spotify_albums dictionary
Album As/Is: Cleveland/Cincinnati, OH - 8/03-8/04/04 songs has been added to spotify_albums dictionary
Album As/Is: Houston, TX - 7/24/04 songs has been added to spotify_albums dictionary
Album As/Is: Philadelphia, PA/Hartford, CT - 8/14-8/15/04 songs has been added to spotify_albums dictionary
Album As/Is: Houston, TX - 7/24/04 songs has been added to spotify_albums dictionary
Album As/Is: Mountain View, CA - 7/16/04 songs has been added to spotify_albums diction

### Get audio features from each song

In [50]:
def audio_features(album):
    #Add new key-values to store audio features
    spotify_albums[album]['acousticness'] = []
    spotify_albums[album]['danceability'] = []
    spotify_albums[album]['energy'] = []
    spotify_albums[album]['instrumentalness'] = []
    spotify_albums[album]['liveness'] = []
    spotify_albums[album]['loudness'] = []
    spotify_albums[album]['speechiness'] = []
    spotify_albums[album]['tempo'] = []
    spotify_albums[album]['valence'] = []
    spotify_albums[album]['popularity'] = []
    #create a track counter
    track_count = 0
    for track in spotify_albums[album]['uri']:
        #pull audio features per track
        features = sp.audio_features(track)
        
        #Append to relevant key-value
        spotify_albums[album]['acousticness'].append(features[0]['acousticness'])
        spotify_albums[album]['danceability'].append(features[0]['danceability'])
        spotify_albums[album]['energy'].append(features[0]['energy'])
        spotify_albums[album]['instrumentalness'].append(features[0]['instrumentalness'])
        spotify_albums[album]['liveness'].append(features[0]['liveness'])
        spotify_albums[album]['loudness'].append(features[0]['loudness'])
        spotify_albums[album]['speechiness'].append(features[0]['speechiness'])
        spotify_albums[album]['tempo'].append(features[0]['tempo'])
        spotify_albums[album]['valence'].append(features[0]['valence'])
        #popularity is stored elsewhere
        pop = sp.track(track)
        spotify_albums[album]['popularity'].append(pop['popularity'])
        track_count+=1

In [51]:
import time
import numpy as np
sleep_min = 2
sleep_max = 5
start_time = time.time()
request_count = 0
for i in spotify_albums:
    audio_features(i)
    request_count+=1
    if request_count % 5 == 0:
        print(str(request_count) + " playlists completed")
        time.sleep(np.random.uniform(sleep_min, sleep_max))
        print('Loop #: {}'.format(request_count))
        print('Elapsed Time: {} seconds'.format(time.time() - start_time))

retrying ...1secs
5 playlists completed
Loop #: 5
Elapsed Time: 15.63957405090332 seconds
10 playlists completed
Loop #: 10
Elapsed Time: 29.60930109024048 seconds
15 playlists completed
Loop #: 15
Elapsed Time: 42.77843379974365 seconds


### Create Dataframe Format Data

In [52]:
dic_df = {}
dic_df['album'] = []
dic_df['track_number'] = []
dic_df['id'] = []
dic_df['name'] = []
dic_df['uri'] = []
dic_df['acousticness'] = []
dic_df['danceability'] = []
dic_df['energy'] = []
dic_df['instrumentalness'] = []
dic_df['liveness'] = []
dic_df['loudness'] = []
dic_df['speechiness'] = []
dic_df['tempo'] = []
dic_df['valence'] = []
dic_df['popularity'] = []
for album in spotify_albums: 
    for feature in spotify_albums[album]:
        dic_df[feature].extend(spotify_albums[album][feature])
        
len(dic_df['album'])

206

In [53]:
import pandas as pd
df = pd.DataFrame.from_dict(dic_df)
df

,album,track_number,id,name,uri,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity
0,The Search for Everything,1,1LM6t24SjQr2bJHqeGIR4U,Still Feel Like Your Man,spotify:track:1LM6t24SjQr2bJHqeGIR4U,0.54700,0.858,0.6000,0.001130,0.1320,-6.677,0.0630,106.023,0.513,68
1,The Search for Everything,2,5ddXMXmXZ2FN4iliTG20nO,Emoji of a Wave,spotify:track:5ddXMXmXZ2FN4iliTG20nO,0.77100,0.494,0.3650,0.000978,0.1390,-8.920,0.0279,165.766,0.191,66
2,The Search for Everything,3,701DK0It9f7iurRnzKvF0y,Helpless,spotify:track:701DK0It9f7iurRnzKvF0y,0.15100,0.761,0.5980,0.002590,0.0314,-5.838,0.0268,103.978,0.937,65
3,The Search for Everything,4,0oiCeOPwm4zhwfyIpAE37y,Love on the Weekend,spotify:track:0oiCeOPwm4zhwfyIpAE37y,0.62300,0.718,0.5830,0.000662,0.0612,-7.884,0.0288,119.970,0.422,70
4,The Search for Everything,5,77Y57qRJBvkGCUw9qs0qMg,In the Blood,spotify:track:77Y57qRJBvkGCUw9qs0qMg,0.38600,0.546,0.6570,0.000010,0.0983,-6.526,0.0255,80.007,0.520,69
5,The Search for Everything,6,6eIygPhGcBFKKcxlGTJlY0,Changing,spotify:track:6eIygPhGcBFKKcxlGTJlY0,0.58700,0.444,0.3360,0.002740,0.1320,-7.778,0.0282,183.345,0.392,63
6,The Search for Everything,7,1G4s7WkhgmdQvYaGCHjw1G,"Theme from ""The Search for Everything""",spotify:track:1G4s7WkhgmdQvYaGCHjw1G,0.89700,0.595,0.3370,0.935000,0.1210,-11.060,0.0263,90.973,0.362,59
7,The Search for Everything,8,1crSsvtU9wZB15dIQXV2QH,Moving On and Getting Over,spotify:track:1crSsvtU9wZB15dIQXV2QH,0.59100,0.850,0.4610,0.040900,0.1000,-9.018,0.0539,94.038,0.628,65
8,The Search for Everything,9,3TQbr3G3U5wlwEJejmqC1F,Never on the Day You Leave,spotify:track:3TQbr3G3U5wlwEJejmqC1F,0.73000,0.535,0.3540,0.000012,0.1380,-8.610,0.0260,139.889,0.220,63
9,The Search for Everything,10,5KsLlcmWDoHUoJFzRw14wD,Rosie,spotify:track:5KsLlcmWDoHUoJFzRw14wD,0.48900,0.758,0.4650,0.000052,0.1160,-6.547,0.0374,138.902,0.759,65


In [55]:
# Remove duplicate data
print(len(df))
final_df = df.sort_values('popularity', ascending=False).drop_duplicates('name').sort_index()
print(len(final_df))

206
166


## Create full list from top 10 artists

In [57]:
# Check the search result for artists
# Change the search index value if the search result is wrong
def check_search_result(artist_list, index_list):
    for i,artist in enumerate(artist_list):
        result = sp.search(artist)
        print(result['tracks']['items'][index_list[i]]['artists'][0]['name'])

In [67]:
def get_artists_uri(artists_list, search_index_list):
    uri_list = []
    for i,artist in enumerate(artists_list):
        result = sp.search(artist)
        uri_list.append(result['tracks']['items'][search_index_list[i]]['artists'][0]['uri'])
    return uri_list

In [74]:
def get_artists_info(uri_list):
    output_list = []
    print('> Adding album information...')
    for uri in uri_list:    
        #Pull all of the artist's albums
        sp_artist = sp.artist(uri)['name']
        print('Added '+sp_artist+' albums ')
        sp_albums = sp.artist_albums(uri, album_type='album')
        album_uris = []
        for i in range(len(sp_albums['items'])):
            album_uris.append(sp_albums['items'][i]['uri'])
        output_list.append({'artist': sp_artist, 'album_uris': album_uris})
    return output_list

In [75]:
def get_track_info(playlist_list):
    for index,artist in enumerate(playlist_list):
        albums_list = {}
        print('> Collecting albums from '+artist['artist']+'....')
        for uri in artist['album_uris']:
            album_name = sp.album(uri)['name']
            albums = {}
            albums['artist'] = [] #create empty list
            albums['album'] = [] 
            albums['track_number'] = []
            albums['id'] = []
            albums['name'] = []
            albums['uri'] = []
            tracks = sp.album_tracks(uri) #pull data on album tracks
            for n in range(len(tracks['items'])): #for each song track
                albums['artist'].append(artist['artist'])
                albums['album'].append(album_name) #append album name tracked via album_count
                albums['track_number'].append(tracks['items'][n]['track_number'])
                albums['id'].append(tracks['items'][n]['id'])
                albums['name'].append(tracks['items'][n]['name'])
                albums['uri'].append(tracks['items'][n]['uri'])
            print('Added '+album_name+' to the albums list')
            albums_list[uri] = albums
        playlist_list[index]['albums'] = albums_list

In [80]:
def audio_features(album):
    album_dict = {}
    #Add new key-values to store audio features
    album_dict['acousticness'] = []
    album_dict['danceability'] = []
    album_dict['energy'] = []
    album_dict['instrumentalness'] = []
    album_dict['liveness'] = []
    album_dict['loudness'] = []
    album_dict['speechiness'] = []
    album_dict['tempo'] = []
    album_dict['valence'] = []
    album_dict['popularity'] = []
    for track in album['uri']:
        #pull audio features per track
        features = sp.audio_features(track)
        
        #Append to relevant key-value
        album_dict['acousticness'].append(features[0]['acousticness'])
        album_dict['danceability'].append(features[0]['danceability'])
        album_dict['energy'].append(features[0]['energy'])
        album_dict['instrumentalness'].append(features[0]['instrumentalness'])
        album_dict['liveness'].append(features[0]['liveness'])
        album_dict['loudness'].append(features[0]['loudness'])
        album_dict['speechiness'].append(features[0]['speechiness'])
        album_dict['tempo'].append(features[0]['tempo'])
        album_dict['valence'].append(features[0]['valence'])
        #popularity is stored elsewhere
        pop = sp.track(track)
        album_dict['popularity'].append(pop['popularity'])
    return album_dict

In [83]:
import time
import random

def get_audio_features(playlist_list):
    start_time = time.time()
    request_count = 0
    print('> Collecting audio features...')
    for index,artist in enumerate(playlist_list):
        for album in artist['albums']:
            features = audio_features(artist['albums'][album])
            playlist_list[index]['albums'][album] = {**playlist_list[index]['albums'][album], **features}
            request_count += 1
            if request_count % 5 == 0:
                print(str(request_count) + " playlists completed")
                time.sleep(random.randint(2,5))
                print('Elapsed Time: {} seconds'.format(time.time() - start_time))
        

In [55]:
artists_list_fadel = ['Tom Misch', 'Barasuara', 'John Mayer', 'Charlie Burg', 'Honne', 'Maliq & D\'Essentials', 'Efek Rumah Kaca', 'Mac Ayres', 'Banda Neira', 'Adhitia Sofyan']
artist_list_ria = ['Rhye', 'Of Monsters and Men', 'Red Hot Chili Peppers', 'foo fighters', 'calvin harris', 'kygo', 'angus and julia stone', 'the national', 'wonderwall']

In [56]:
artists_search_index_list_fadel = [2,1,2,1,2,1,1,1,1,1]
artists_search_index_list_ria = [1,1,1,1,1,1,1,1,1,1]

In [85]:
check_search_result(artist_list_ria, artists_search_index_list_ria)

Rhye
Of Monsters and Men
Red Hot Chili Peppers
Foo Fighters
Calvin Harris
Kygo
Angus & Julia Stone
The National
Oasis


In [86]:
artists_uri_list_ria = get_artists_uri(artist_list_ria, artists_search_index_list_ria)
artists_uri_list_ria

['spotify:artist:2AcUPzkVWo81vumdzeLLRN',
 'spotify:artist:4dwdTW1Lfiq0cM8nBAqIIz',
 'spotify:artist:0L8ExT028jH3ddEcZwqJJ5',
 'spotify:artist:7jy3rLJdDQY21OgRLCZ9sD',
 'spotify:artist:7CajNmpbOovFoOoasH2HaY',
 'spotify:artist:23fqKkggKUBHNkbKtXEls4',
 'spotify:artist:4tvKz56Tr39bkhcQUTO0Xr',
 'spotify:artist:2cCUtGK9sDU2EoElnk0GNB',
 'spotify:artist:2DaxqgrOhkeH0fpeiQq2f4']

In [87]:
spotify_playlist = get_artists_info(artists_uri_list_ria)

> Adding album information...
Added Rhye albums 
Added Of Monsters and Men albums 
Added Red Hot Chili Peppers albums 
Added Foo Fighters albums 
Added Calvin Harris albums 
Added Kygo albums 
Added Angus & Julia Stone albums 
Added The National albums 
Added Oasis albums 


In [88]:
get_track_info(spotify_playlist)

> Collecting albums from Rhye....
Added Spirit to the albums list
Added Spirit to the albums list
Added Spirit to the albums list
Added Blood Remixed to the albums list
Added Blood Remixed to the albums list
Added Blood Remixed to the albums list
Added Blood Remixed (Pt. 2) to the albums list
Added Blood Remixed (Pt. 2) to the albums list
Added Blood to the albums list
Added Blood to the albums list
Added Blood to the albums list
Added Blood to the albums list
Added Woman to the albums list
Added Woman to the albums list
Added Woman to the albums list
> Collecting albums from Of Monsters and Men....
Added FEVER DREAM to the albums list
Added FEVER DREAM to the albums list
Added Fever Dream to the albums list
Added Beneath The Skin (Deluxe) to the albums list
Added Beneath the Skin to the albums list
Added Beneath The Skin to the albums list
Added LIVE FROM VATNAGARÐAR to the albums list
Added My Head Is An Animal to the albums list
Added My Head Is An Animal to the albums list
Added My

In [89]:
get_audio_features(spotify_playlist)

> Collecting audio features...
retrying ...2secs
retrying ...1secs
5 playlists completed
Elapsed Time: 12.313785791397095 seconds
10 playlists completed
Elapsed Time: 23.00599193572998 seconds
15 playlists completed
Elapsed Time: 32.98318290710449 seconds
retrying ...1secs
20 playlists completed
Elapsed Time: 45.9645299911499 seconds
25 playlists completed
Elapsed Time: 57.44316506385803 seconds
30 playlists completed
Elapsed Time: 73.33332371711731 seconds
35 playlists completed
Elapsed Time: 88.60248613357544 seconds
retrying ...2secs
retrying ...1secs
40 playlists completed
Elapsed Time: 105.37648582458496 seconds
45 playlists completed
Elapsed Time: 117.18835306167603 seconds
50 playlists completed
Elapsed Time: 130.7137291431427 seconds
retrying ...1secs
55 playlists completed
Elapsed Time: 142.38905096054077 seconds
retrying ...1secs
60 playlists completed
Elapsed Time: 158.21052980422974 seconds
retrying ...3secs
retrying ...1secs
65 playlists completed
Elapsed Time: 173.9801130

In [90]:
dic_df = {}
dic_df['artist'] = []
dic_df['album'] = []
dic_df['track_number'] = []
dic_df['id'] = []
dic_df['name'] = []
dic_df['uri'] = []
dic_df['acousticness'] = []
dic_df['danceability'] = []
dic_df['energy'] = []
dic_df['instrumentalness'] = []
dic_df['liveness'] = []
dic_df['loudness'] = []
dic_df['speechiness'] = []
dic_df['tempo'] = []
dic_df['valence'] = []
dic_df['popularity'] = []
dic_df['artist'] = []

for artist in spotify_playlist:
    for album in artist['albums']: 
        for feature in artist['albums'][album]:
            dic_df[feature].extend(artist['albums'][album][feature])
        
len(dic_df['album'])

1859

In [91]:
import pandas as pd

df = pd.DataFrame.from_dict(dic_df)
df

,artist,album,track_number,id,name,uri,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity
0,Rhye,Spirit,1,08izEDWoClxZxwayRzSv6s,Dark,spotify:track:08izEDWoClxZxwayRzSv6s,0.995000,0.375,0.0966,0.944000,0.1110,-18.421,0.0316,102.641,0.0406,45
1,Rhye,Spirit,2,5MHHcFRlO9LUP153GFHOk6,Needed,spotify:track:5MHHcFRlO9LUP153GFHOk6,0.861000,0.678,0.0611,0.099100,0.1130,-19.514,0.0335,148.008,0.1800,57
2,Rhye,Spirit,3,59LQkFxbgLiSJs6R6rZZ9e,Malibu Nights,spotify:track:59LQkFxbgLiSJs6R6rZZ9e,0.990000,0.541,0.0799,0.942000,0.1110,-27.832,0.0335,121.940,0.1330,46
3,Rhye,Spirit,4,5pW62tsMUPtw8k69ifpcjJ,Patience,spotify:track:5pW62tsMUPtw8k69ifpcjJ,0.957000,0.631,0.1400,0.010600,0.1090,-16.376,0.0368,139.961,0.1000,53
4,Rhye,Spirit,5,7FI6YrnHlJKPgK1VOHbdEV,Green Eyes,spotify:track:7FI6YrnHlJKPgK1VOHbdEV,0.954000,0.415,0.1570,0.838000,0.1090,-20.237,0.0305,107.643,0.0376,47
5,Rhye,Spirit,6,6dvHjr6kOqEnKeQyd3Tozs,Wicked Dreams,spotify:track:6dvHjr6kOqEnKeQyd3Tozs,0.888000,0.760,0.2690,0.524000,0.0838,-14.264,0.0329,141.050,0.4720,54
6,Rhye,Spirit,7,3hacWElpudxiFs4sPDYKu9,Awake,spotify:track:3hacWElpudxiFs4sPDYKu9,0.958000,0.234,0.0312,0.791000,0.1000,-21.700,0.0413,112.399,0.0551,43
7,Rhye,Spirit,8,3N0YfkkYU7UZPiFZqoUUsM,Save Me,spotify:track:3N0YfkkYU7UZPiFZqoUUsM,0.985000,0.348,0.1160,0.013600,0.0972,-17.768,0.0401,163.692,0.1910,46
8,Rhye,Spirit,1,7sdoPUtu8pN1YuMilAzNEn,Dark,spotify:track:7sdoPUtu8pN1YuMilAzNEn,0.995000,0.375,0.0966,0.944000,0.1110,-18.421,0.0316,102.641,0.0406,26
9,Rhye,Spirit,2,21v43BXLzfeMNDovkL4hcR,Needed,spotify:track:21v43BXLzfeMNDovkL4hcR,0.861000,0.678,0.0611,0.099100,0.1130,-19.514,0.0335,148.008,0.1800,37


In [92]:
# Remove duplicate data
print(len(df))
final_df = df.sort_values('popularity', ascending=False).drop_duplicates('name').sort_index()
print(len(final_df))

1859
949


In [93]:
from openpyxl import load_workbook

path = r"./playlist_fadel.xlsx"

book = load_workbook(path)
writer = pd.ExcelWriter(path, engine = 'openpyxl')
writer.book = book

final_df.to_excel(writer, sheet_name = 'playlist_ria')
writer.save()
writer.close()